In [2]:
!pip install --upgrade fastcore -q
!pip install --upgrade fastai -q

^C
ERROR: Operation cancelled by user


In [ ]:
from fastai.vision.all import *

In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans


In [1]:
crashes = pd.read_csv('Train.csv')
crashes.head()

NameError: name 'pd' is not defined

In [ ]:
kmeans = KMeans(n_clusters = 6)
cols = ['datetime', 'latitude', 'longitude']
X = crashes[cols]
kmeans.fit(X[X.columns[1:3]])
X['cluster_labels'] = kmeans.fit_predict(X[X.columns[1:3]])
centers = kmeans.cluster_centers_
centroids = centers.copy()
centers = torch.from_numpy(centers)
centers

In [ ]:
ambulance_loc = centers.clone()
#ambulance_loc = torch.from_numpy(ambulance_loc)
ambulance_loc.requires_grad_()
crash_locations = tensor(crashes[['latitude', 'longitude']].values)
#define loss function 
def loss_fn(crash_locations, ambulance_loc):
  """For each crash we find the dist to the closest ambulance, and return the mean of these dists."""
  # Dists to first ambulance
  dists_split = crash_locations-ambulance_loc[0]
  dists = (dists_split[:,0]**2 + dists_split[:,1]**2)**0.5
  min_dists = dists
  for i in range(1, 6):
    # Update dists so they represent the dist to the closest ambulance
    dists_split = crash_locations-ambulance_loc[i]
    dists = (dists_split[:,0]**2 + dists_split[:,1]**2)**0.5
    min_dists = torch.min(min_dists, dists)
  
  return min_dists.mean()

In [ ]:
computed_loss = loss_fn(crash_locations, ambulance_loc)
computed_loss


In [ ]:
#the gradients can be computed as follows:
computed_loss.backward()
ambulance_loc.grad

In [ ]:
#the loss can be reduced based on inferences from the gradients above as follows: 
ambulance_loc.grad = None # Reset gradients
lr=1e-2 # Pick LR
for i in range(1000):
  computed_loss = loss_fn(crash_locations, ambulance_loc) # Find loss
  if i % 100 == 0: print('Loss:', computed_loss.item()) # Print loss
  computed_loss.backward() # Calc grads
  ambulance_loc.data -= lr * ambulance_loc.grad.data # Update locs
  ambulance_loc.grad = None

In [ ]:
# Plot crashes and ambulance locations
ax = crashes.plot(y='latitude', x='longitude', kind='scatter', figsize=(16, 10), label='crash locations')
ax.scatter(centers[:,1].detach().numpy(), ambulance_loc[:,0].detach().numpy(), c='red', label='initial ambulance locs')
ax.scatter(ambulance_loc[:,1].detach().numpy(), ambulance_loc[:,0].detach().numpy(), c='yellow', label='final ambulance locs')
ax.legend()

In [ ]:
#Performing the SGD
# Split for validation
train_locations = crash_locations[:-1000]
val_locations = crash_locations[-1000:]

# Load crash locs from train into a dataloader
dl = DataLoader(train_locations, batch_size=1000
                , shuffle=True)

# Set up ambulance locations
ambulance_locations = centers
ambulance_locations.requires_grad_()

# Set vars
lr=3e-3
n_epochs = 70
mom = 0.9
# Store loss over time
train_losses = []
val_losses = []

# Training loop
for epoch in range(n_epochs):
   # Run through batches
  for crashes in dl:
    loss = loss_fn(crashes, ambulance_locations) # Find loss for this batch of crashes
    loss.backward() # Calc grads
    ambulance_locations.data -= lr * ambulance_locations.grad.data # Update locs
    ambulance_locations.grad = None # Reset gradients for next step
    train_losses.append(loss.item())
    val_losses.append(loss_fn(val_locations, ambulance_locations).item()) # Can remove as this lows things down

  # Print validation loss
  print('Val loss:', loss_fn(val_locations, ambulance_locations).item())

In [ ]:
ambulance_locations

In [ ]:
ambulance_locations
    
    
    
        
    

In [ ]:
X.cluster_labels.unique()

In [ ]:
centroids

In [ ]:
dates = pd.date_range('2019-01-01', '2020-01-01', freq='3h')
submission = pd.DataFrame({'date':dates})
submission.head()

In [ ]:
ambulance_locations

def random_a_0_latitudes(initial_loc, final_loc, num): 
    res = [] 
  
    for j in range(num): 
        res.append(random.uniform(initial_loc, final_loc)) 
  
    return res
A0_lat = random_a_0_latitudes(-1.2640,-1.2643, len(submission))
A0_long = random_a_0_latitudes(36.7450, 36.7452, len(submission))
A1_lat = random_a_0_latitudes(-1.4707, -1.4710, len(submission))
A1_long = random_a_0_latitudes(37.0657, 37.0659, len(submission))
A2_lat = random_a_0_latitudes(-2.2664, -2.2666, len(submission))
A2_long = random_a_0_latitudes(37.3877, 37.3885, len(submission))
A3_lat = random_a_0_latitudes(-1.0360, -1.0365, len(submission))
A3_long = random_a_0_latitudes(37.0601, 37.0621, len(submission))
A4_lat = random_a_0_latitudes(-1.2979, -1.2981, len(submission))
A4_long = random_a_0_latitudes(36.8372, 36.8375, len(submission))
A5_lat = random_a_0_latitudes(-1.2175, -1.2178, len(submission))
A5_long = random_a_0_latitudes(36.8961, 36.8965,  len(submission))
submission['A0'+'_Latitude'] = A0_lat
submission['A0'+'_Longitude'] = A0_long
submission['A1'+'_Latitude'] = A1_lat
submission['A1'+'_Longitude'] = A1_long
submission['A2'+'_Latitude'] = A2_lat
submission['A2'+'_Longitude'] = A2_long
submission['A3'+'_Latitude'] = A3_lat
submission['A3'+'_Longitude'] = A3_long
submission['A4'+'_Latitude'] = A4_lat
submission['A4'+'_Longitude'] = A4_long
submission['A5'+'_Latitude'] = A5_lat
submission['A5'+'_Longitude'] = A5_long
submission.head()

In [ ]:
submission.to_csv('submission_file1.csv')

In [ ]:
ambulance_locations

In [ ]:
ambulance_locations = ambulance_locations.detach().numpy()


In [ ]:
dates = pd.date_range('2019-01-01', '2020-01-01', freq='3h')
sub = pd.DataFrame({
    'date':dates
})
sub['A0'+'_Latitude'] = ambulance_locations[0,0]
sub['A0'+'_Longitude'] = ambulance_locations[0][1]
sub['A1'+'_Latitude'] = ambulance_locations[1][0]
sub['A1'+'_Longitude'] = ambulance_locations[1][1]
sub['A2'+'_Latitude'] = ambulance_locations[2][0]
sub['A2'+'_Longitude'] = ambulance_locations[2][1]
sub['A3'+'_Latitude'] = ambulance_locations[3][0]
sub['A3'+'_Longitude'] = ambulance_locations[3][1]
sub['A4'+'_Latitude'] = ambulance_locations[4][0]
sub['A4'+'_Longitude'] = ambulance_locations[4][1]
sub['A5'+'_Latitude'] = ambulance_locations[5][0]
sub['A5'+'_Longitude'] = ambulance_locations[5][1]
'''for ambulance in range(1, 6):
    sub['A'+str(ambulance)+'_Latitude'] = ambulance_locationa[0][0]
    sub['A'+str(ambulance)+'_Longitude'] = 0

# Place an ambulance in the center of the city:
sub['A'+str(ambulance)+'_Latitude'] = 36.82
sub['A'+str(ambulance)+'_Longitude'] = -1.3
'''

sub.head()